In [394]:
import numpy as np
import pandas as pd
!git clone https://github.com/tsebaka/ML-from-scratch
import sys
sys.path.insert(0,'/content/ML-from-scratch/Parse-lib')
import numpy as np
import nltk
import bs4 as bs
import re
import urllib.request
import warnings
import pp

from nltk.corpus import stopwords

warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('punkt')

fatal: destination path 'ML-from-scratch' already exists and is not an empty directory.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [395]:
sites =[#'https://breakingbad.fandom.com/wiki/Walter_White',
         'https://en.wikipedia.org/wiki/Breaking_Bad',
        #  'https://breakingbad.fandom.com/wiki/Jesse_Pinkman',
        #  'https://breakingbad.fandom.com/wiki/Gustavo_Fring', 
        # 'https://breakingbad.fandom.com/wiki/Jimmy_McGill',
        # 'https://breakingbad.fandom.com/wiki/Mike_Ehrmantraut',
        # 'https://breakingbad.fandom.com/wiki/Skyler_White',
        # 'https://breakingbad.fandom.com/wiki/Hank_Schrader'
         ]

parser = pp.parse_and_prepare(EOS=True)
vocab, text, all_word = parser.parse(sites)

In [396]:
import gensim

model = gensim.models.Word2Vec(all_word,
                              min_count=1, 
                              workers=4,
                              size=50,
                              window=3,
                              iter=10)

In [397]:
model.wv['jesse']

array([-0.80519515, -0.01865546,  0.34359923,  0.4018504 , -0.01313855,
       -0.18804446,  0.09777899,  0.03102148, -0.04293525,  0.02108033,
       -0.08953659, -0.12652211, -0.31095377,  0.21592027,  0.10771915,
        0.42813733, -0.05000349,  0.22815248,  0.29980892, -0.27068007,
        0.35995916,  0.18484813, -0.03201825, -0.12530313,  0.47517127,
       -0.12698232, -0.07086036,  0.17725305,  0.26749054, -0.5129367 ,
        0.23906566,  0.3105898 , -0.25617987,  0.23161663, -0.08303642,
       -0.3416912 , -0.06867372,  0.31836742, -0.14585724,  0.06097819,
        0.13171798,  0.17051606,  0.18732682,  0.2501148 , -0.38491118,
        0.2656173 , -0.02340393, -0.12001038,  0.10801007, -0.6975137 ],
      dtype=float32)

In [398]:
from collections import defaultdict

In [399]:
vocab = np.unique(text)

In [400]:
diction = defaultdict(lambda: len(vocab))
idx_to_word = defaultdict(lambda: len(vocab))

for idx, word in enumerate(vocab):
    diction[word] = idx
    idx_to_word[idx] = word

In [401]:
idx_to_word[2193]

'walt'

In [402]:
diction['walt']

2193

In [403]:
all_word[0]

['breaking',
 'bad',
 'is',
 'an',
 'american',
 'crime',
 'drama',
 'television',
 'series',
 'created',
 'and',
 'produced',
 'by',
 'vince',
 'gilligan',
 '.']

In [404]:
data = []

In [405]:
all_word[0]

['breaking',
 'bad',
 'is',
 'an',
 'american',
 'crime',
 'drama',
 'television',
 'series',
 'created',
 'and',
 'produced',
 'by',
 'vince',
 'gilligan',
 '.']

In [406]:
for i in range(len(all_word)):
    target = []
    for j in range(len(all_word[i]) - 1):
        target.append(all_word[i][j + 1])

    all_word[i].pop()
    data.append((all_word[i], target))

In [407]:
len(data)

425

In [408]:
validation_set = data[350:425]
training_set = data[0:350]

In [409]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Recurrent layer
        # YOUR CODE HERE!
        self.lstm = nn.LSTM(input_size=len(vocab) + 1,
                         hidden_size=50,
                         num_layers=1,
                         bidirectional=False)
        
        # Output layer
        self.l_out = nn.Linear(in_features=50,
                            out_features=len(vocab) + 1,
                            bias=False)
        
    def forward(self, x):
        # RNN returns output and last hidden state
        x, (h, c) = self.lstm(x)
        
        # Flatten output for feed-forward layer
        x = x.view(-1, self.lstm.hidden_size)
        
        # Output layer
        x = self.l_out(x)
        
        return x

net = Net()
print(net)

Net(
  (lstm): LSTM(2267, 50)
  (l_out): Linear(in_features=50, out_features=2267, bias=False)
)


In [410]:
def one_hot_encode(idx, vocab_size):
    """
    One-hot encodes a single word given its index and the size of the vocabulary.
    
    Args:
     `idx`: the index of the given word
     `vocab_size`: the size of the vocabulary
    
    Returns a 1-D numpy array of length `vocab_size`.
    """
    # Initialize the encoded array
    one_hot = np.zeros(vocab_size)
    
    # Set the appropriate element to one
    one_hot[idx] = 1.0

    return one_hot
def one_hot_encode_sequence(sequence, vocab_size):
    """
    One-hot encodes a sequence of words given a fixed vocabulary size.
    
    Args:
     `sentence`: a list of words to encode
     `vocab_size`: the size of the vocabulary
     
    Returns a 3-D numpy array of shape (num words, vocab size, 1).
    """
    # Encode each word in the sentence
    encoding = np.array([one_hot_encode(diction[word], vocab_size) for word in sequence])

    # Reshape encoding s.t. it has shape (num words, vocab size, 1)
    encoding = encoding.reshape(encoding.shape[0], encoding.shape[1], 1)
    
    return encoding

In [411]:
# Hyper-parameters
num_epochs = 50

# Initialize a new network
net = Net()

# Define a loss function and optimizer for this problem
# YOUR CODE HERE!
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)

# Track loss
training_loss, validation_loss = [], []

# For each epoch
for i in range(num_epochs):
    
    # Track loss
    epoch_training_loss = 0
    epoch_validation_loss = 0
    
    net.eval()
    cnt = 0
    # For each sentence in validation set
    for inputs, targets in validation_set:
        # One-hot encode input and target sequence
        # embeds = []
        # for word in inputs:
        #     embeds.append(model.wv[word].transpose())
        # embeds = np.array(embeds)
        # embeds = embeds.transpose()
        # embeds = np.array([embeds])
        embeds = []
        for k in range(len(inputs)):
            one_hot = [0] * (len(vocab) + 1)
            # print(inputs[k])
            one_hot[diction[inputs[k]]] = 1
            embeds.append(one_hot)
        embeds = np.array(embeds)
        embeds = embeds.transpose()
        embeds = np.array([embeds])
        targets_idx = [diction[word] for word in targets]
        
        embeds = torch.Tensor(embeds)
        # print(embeds.shape)
        embeds = embeds.permute(0, 2, 1)
        targets_idx = torch.LongTensor(targets_idx)

        outputs = net.forward(embeds)

        loss = criterion(outputs, targets_idx)

        epoch_validation_loss += loss.detach().numpy()
    
    net.train()
    
    # For each sentence in training set
    cnt = 0
    for inputs, targets in training_set:
        # cnt += 1
        # if cnt < 276:
        # One-hot encode input and target sequence
        embeds = []
        for k in range(len(inputs)):
            one_hot = [0] * (len(vocab) + 1)
            # print(inputs[k])
            one_hot[diction[inputs[k]]] = 1
            embeds.append(one_hot)
        embeds = np.array(embeds)
        embeds = embeds.transpose()
        embeds = np.array([embeds])
        targets_idx = [diction[word] for word in targets]
        
        # Convert target to tensor
        embeds = torch.Tensor(embeds)
        embeds = embeds.permute(0, 2, 1)
        targets_idx = torch.LongTensor(targets_idx)
        # Forward pass
        
        # YOUR CODE HERE!
        outputs = net.forward(embeds)
        
        # Compute loss
        # YOUR CODE HERE!

        loss = criterion(outputs, targets_idx)
        
        # Backward pass
        # YOUR CODE HERE!
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update loss
        epoch_training_loss += loss.detach().numpy()
        # else:
        #     break
        
    # Save loss for plot
    training_loss.append(epoch_training_loss/len(training_set))
    validation_loss.append(epoch_validation_loss/len(validation_set))

    # Print loss every 5 epochs
    if i % 5 == 0:
        print(f'Epoch {i}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')


Epoch 0, training loss: 7.515769438062395, validation loss: 7.7183026059468585
Epoch 5, training loss: 4.614141505445753, validation loss: 5.108729257583618
Epoch 10, training loss: 4.525316413130079, validation loss: 5.206770779291789
Epoch 15, training loss: 4.437944059712546, validation loss: 5.296433534622192
Epoch 20, training loss: 4.346894176006317, validation loss: 5.381518402099609
Epoch 25, training loss: 4.255597752843585, validation loss: 5.491906579335531
Epoch 30, training loss: 4.150505072729928, validation loss: 5.624347235361735
Epoch 35, training loss: 4.038934882708959, validation loss: 5.764317282040914
Epoch 40, training loss: 3.932171310697283, validation loss: 5.891244384447734
Epoch 45, training loss: 3.8311471029690334, validation loss: 5.991313852469126


In [412]:
diction['is']

2266

In [413]:
idx_to_word[2266]

2266

In [423]:
# Get first sentence in test set
inputs, targets = validation_set[1]
targets = ['cranston']
inputs = ['bryan']
# One-hot encode input and target sequence
embeds = []
for word in inputs:
    embeds.append(model.wv[word])
targets_idx = [diction[word] for word in targets]

# Convert input to tensor
embeds = []
for k in range(len(inputs)):
    one_hot = [0] * (len(vocab) + 1)
    # print(inputs[k])
    one_hot[diction[inputs[k]]] = 1
    embeds.append(one_hot)
embeds = np.array(embeds)
embeds = embeds.transpose()
embeds = np.array([embeds])
targets_idx = [diction[word] for word in targets]

# Convert target to tensor
embeds = torch.Tensor(embeds)
embeds = embeds.permute(0, 2, 1)
targets_idx = torch.LongTensor(targets_idx)

# Forward pass
# YOUR CODE HERE!
outputs = net.forward(embeds).data.numpy()

print('\nInput sequence:')
print(inputs)

print('\nTarget sequence:')
print(targets)

print('\nPredicted sequence:')
# print([idx_to_word[np.argmax(outputs)] for output in outputs])
for i in range(len(outputs)):
    print(idx_to_word[np.argmax(outputs[i][0:2260])])
# Plot training and validation loss


Input sequence:
['bryan']

Target sequence:
['cranston']

Predicted sequence:
cranston
